In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
df= pd.read_csv("sap_storing_data_hu_project.csv")

In [ ]:
df.head()

haalt de data's uithalen dat de Betekenis een vraagteken of nvt staan.

In [ ]:
df=df[['#stm_sap_meldnr','stm_mon_nr','stm_vl_post','stm_sap_meld_ddt','stm_sap_meldtekst_lang','stm_sap_meldtekst','stm_geo_mld','stm_equipm_nr_mld','stm_equipm_soort_mld','stm_equipm_omschr_mld','stm_km_van_mld',
   'stm_km_tot_mld','stm_prioriteit','stm_aanngeb_ddt','stm_oh_pg_gst','stm_geo_gst','stm_equipm_nr_gst','stm_equipm_soort_gst','stm_equipm_omschr_gst','stm_km_van_gst','stm_km_tot_gst','stm_oorz_groep','stm_oorz_code',
   'stm_oorz_tkst','stm_fh_ddt','stm_fh_status','stm_sap_storeind_ddt','stm_tao_indicator','stm_tao_indicator_vorige','stm_tao_soort_mutatie','stm_tao_telling_mutatie','stm_tao_beinvloedbaar_indicator','stm_sap_melddatum',
   'stm_sap_meldtijd','stm_contractgeb_mld','stm_techn_mld','stm_techn_gst','stm_aanngeb_dd','stm_aanngeb_tijd','stm_aanntpl_dd','stm_aanntpl_tijd','stm_progfh_in_datum','stm_progfh_in_tijd','stm_progfh_in_invoer_dat',
   'stm_progfh_in_invoer_tijd','stm_progfh_in_duur','stm_progfh_gw_tijd','stm_progfh_gw_teller','stm_fh_dd','stm_fh_tijd','stm_fh_duur','stm_sap_storeinddatum','stm_sap_storeindtijd','stm_oorz_tekst_kort',
   'stm_pplg_van','stm_pplg_naar','stm_dstrglp_van','stm_dstrglp_naar']]

In [ ]:
df.info()

filter functiehersteltijd met bijhoorde oorzaak code voor de baseline model 

# data understanding en eerste filtering

In [ ]:
null = df.loc[df['stm_fh_duur']==0,['stm_fh_duur']]
vijftien = df.loc[ (df['stm_fh_duur'] >0) & (df['stm_fh_duur']<=15 ),['stm_fh_duur']]
vijftien_halfuur = df.loc[ (df['stm_fh_duur'] >15) & (df['stm_fh_duur']<=30 ),['stm_fh_duur']]
halfuur_uur = df.loc[( df['stm_fh_duur']> 30) & (df['stm_fh_duur']<=60) ,['stm_fh_duur']]
een_twee_uur = df.loc[ (df['stm_fh_duur']>60) & (df['stm_fh_duur']<=120) ,['stm_fh_duur']]
twee_drie_uur = df.loc[ (df['stm_fh_duur']>120) & (df['stm_fh_duur']<=180) ,['stm_fh_duur']]
drie_zes_uur = df.loc[ (df['stm_fh_duur']>180) & (df['stm_fh_duur']<=360) ,['stm_fh_duur']]
zes_acht_uur = df.loc[ (df['stm_fh_duur']>360) & (df['stm_fh_duur']<=540) ,['stm_fh_duur']]
acht_plus = df.loc[df['stm_fh_duur']> 540,['stm_fh_duur']]
acht_plus = df.loc[df['stm_fh_duur']> 540,['stm_fh_duur']]
print(f'socre null: {len(null)/len(df)}')
print(f'socre tot 15m: {len(vijftien)/len(df)}')
print(f'socre 15m tot 30m: {len(vijftien_halfuur)/len(df)}')
print(f'socre 30m tot 1h: {len(halfuur_uur)/len(df)}')
print(f'socre 1h tot 2h: {len(een_twee_uur)/len(df)}')
print(f'socre 2h tot 3h: {len(twee_drie_uur)/len(df)}')
print(f'socre 3h tot 6h: {len(drie_zes_uur)/len(df)}')
print(f'socre 6h tot 8h: {len(zes_acht_uur)/len(df)}')
print(f'socre 8h+: {len(acht_plus)/len(df)}')

we zien dat dat 19% van de meldingen geen probleem zijn dat de spoor op stil staat <br>
8% dat onder de 15 miuten is en 4% tussen 15-30 min <br>
voor meldingen dat de spoor boven de acht uurr stil staan zij er maar 17%

In [ ]:
len(df.loc[df['stm_fh_duur']> 1440,['stm_fh_duur']])/len(df)

7.5% van alle data is het spoor boven een dag stilgestaan

we dorpen data's dat we niet kunnen op getraint kan

In [ ]:
df2 = df.dropna(subset=['stm_fh_duur'])

filter data's dat stm_fh_duur onder de 15 min en data's boven de 8 uur,het is niet belangrjik in deze project.<br>
we nemen data's van 15min tot 8uur, zo data de model niet telaag gaat onderschaten.

In [ ]:
df2 = df2.loc[ (df['stm_fh_duur']>15) & (df['stm_fh_duur']<=540)]

we hebben nog 65% data over voor de model

In [ ]:
len(df2)/len(df)

hier zijn wat feature dat we belangrijk vinden:<br>
stm_oorz_code : code van wat de probleem is (weet pas was de aannemer aanwezig is) <br>
stm_sap_melddatum: de datum data melding melding is binnen gekommen <br>
stm_sap_meldtijd : de tijdstip van de melding <br>
stm_geo_mld: geocode waar iets gebuurt is. <br>
stm_sap_meldtijd : de datum dat de aanmener anweizig is （het is binnen 8uur het kan zijn dat dit niet van belang is） <br>
stm_aanntpl_tijd : de tijdmoment dat de aanmener anweizig is <br>
stm_fh_dd : de datum dat de probleem opgelost is <br>
stm_aanntpl_tijd : de tijdmoment dat de probleem opgelost is （het is binnen 8uur het kan zijn dat dit niet van belang is） <br>
stm_techn_mld: ? (gekrijgen van docent)<br>
stm_prioriteit: ?(gekrijgen van docent)<br>
stm_fh_duur: hoelang het duurt van melding tot dat de probleem is opgelost

In [ ]:
df2=df2[['stm_oorz_code','stm_sap_melddatum','stm_sap_meldtijd','stm_geo_mld','stm_aanntpl_dd','stm_aanntpl_tijd','stm_fh_dd','stm_fh_tijd','stm_techn_mld','stm_prioriteit','stm_fh_duur']]

In [ ]:
df2.info()

# eerste data preperatie

In [ ]:
df2[['stm_sap_meldtijd','stm_aanntpl_tijd','stm_fh_tijd']]

stm_sap_meldtijd, stm_aanntpl_tijd en stm_fh_tijd is een object dat geschikt is voor model <br>
hieronder woordt het omgezet naar welkste minuut het van de dag is als int

In [ ]:
# naar datetime tpye 
df2['stm_sap_meldtijd'] = pd.to_datetime(df2['stm_sap_meldtijd'], format='%H:%M:%S', errors='coerce')
# verandert het h:min naar totale minuten
df2['stm_sap_meldtijd'] = df2['stm_sap_meldtijd'].apply(lambda x: x.hour*60 + x.minute)

df2['stm_aanntpl_tijd'] = pd.to_datetime(df2['stm_aanntpl_tijd'], format='%H:%M:%S', errors='coerce')
df2['stm_aanntpl_tijd'] = df2['stm_aanntpl_tijd'].apply(lambda x: x.hour*60 + x.minute)

df2['stm_fh_tijd'] = pd.to_datetime(df2['stm_fh_tijd'], format='%H:%M:%S', errors='coerce')
df2['stm_fh_tijd'] = df2['stm_fh_tijd'].apply(lambda x: x.hour*60 + x.minute)

In [ ]:
df2=df2.loc[df2['stm_aanntpl_tijd']!=0]

In [ ]:
df2

stm_sap_melddatum is een ook object dat geschikt is voor modellen <br>
hieronder woordt het omgezet naar welkste dag het van de jaar is als int <br>

In [ ]:
# naar datetime tpye 
df2['stm_sap_melddatum'] = pd.to_datetime(df2['stm_sap_melddatum'], format='%d/%m/%Y')
# kiijk welkeste dag het van de jaar is
df2['stm_sap_melddatum'] = df2['stm_sap_melddatum'].dt.dayofyear

# target

voor de traget neem we de tijd neem de functie hersteltijd van waarneer de aanmener aanweizig is tot dat weer gereiden mogen worden.<br>
voor de dat stm_sap_meldtijd 0 is neem we stm_fh_tijd 

In [ ]:
df2['targetherstel'] = np.where(
    df2['stm_fh_tijd'] - df2['stm_aanntpl_tijd']>= 0, # als de stm_fh_tijd-stm_aanntpl_tijd positief is
    df2['stm_fh_tijd'] - df2['stm_aanntpl_tijd'], # dan betekende dat nog op de zefde day is, kun je gelijk van elkaar aftrekken
    df2['stm_fh_tijd'] - df2['stm_aanntpl_tijd']+24 # als niet op zelfdde dag is hersteld, dus een neagetief uitkomst moet er +24 om de juiste tijd te krijgen.
)

In [ ]:
df2=df2[df2['targetherstel']!=0]

# baseline model

we nemen hier de gemiddelde hersteltijd van alle stm_oorz_code als onze voorspelling, we neem 10% marge als de werkelijk waarde in de marge zit dan reken we het als goed.
(waarneem voor deze baseline model dat altijd de oorzaakcode bekend is)

In [ ]:
# om het gebruik maakt zet maken voor de balk code onder deze balk
# is hier een dict gevormt met de oorzaakcode als key en gemiddelde hersteltijd als value
df_gm=df2.groupby('stm_oorz_code')['targetherstel'].agg(['mean'])# sorten bij stm_oorz_code waar targetherstel gemiddelde word genomen
df_value=df_gm['mean'].to_list() #zet alle gemidde hersteltijd als value in een list
df_key=df2['stm_oorz_code'].unique()  #zet alle oorzaakcode als value in een list (elk oorzaakcode komt niet tweekeer voor)
dictionary = dict(zip(df_key, df_value)) # met zip zet we om in tot een dict

In [ ]:
cnt=0
        
for index, row in df.iterrows(): #lopen door de data
    key = row["stm_oorz_code"] # kijk wat de huidig oorzaakcode is
    value_in_dict = dictionary.get(key, None) # zoek de gemiddelde functie hersteltijd op van deze oorzaakcode op
    if value_in_dict is not None:
        if row["stm_fh_duur"] > value_in_dict*0.9 and row["stm_fh_duur"] < value_in_dict*1.1: # als werlkelijk waarde tussen de 10% marge van de gemiddelde hersteltijd is
            cnt+=1 # dan tel als een goed
cnt/(len(df2) - df2['stm_oorz_code'].isna().sum() )*100 #het score in precentage goed is: de totaal goed dellen door de de totale lengte van de data dat een oorzaakcode heeft. 

# model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [ ]:
df2.info()

In [ ]:
df2['stm_sap_meldtijd']=df2['stm_sap_meldtijd'].fillna('0')

In [ ]:
df2[['stm_techn_mld']]=df2[['stm_techn_mld']].fillna('0')
unique_values = df2['stm_techn_mld'].unique()
techn_mld_dict = {key: index + 1 for index, key in enumerate(unique_values)}
df2['stm_techn_mld'] = df2['stm_techn_mld'].apply(lambda x: techn_mld_dict[x])

In [ ]:
df2[['stm_geo_mld']]=df2[['stm_geo_mld']].fillna('0')
unique_values = df2['stm_geo_mld'].unique()
stm_geo_mld_dict = {key: index + 1 for index, key in enumerate(unique_values)}
df2['stm_geo_mld'] = df2['stm_geo_mld'].apply(lambda x: stm_geo_mld_dict[x])

In [ ]:
df2[['stm_oorz_code']]= df2[['stm_oorz_code']].fillna('0')
df2['stm_oorz_code']=df2['stm_oorz_code'].astype('int32')

In [ ]:
df2['stm_prioriteit']=df2['stm_prioriteit'].fillna('0')
df2['stm_prioriteit']=df2['stm_prioriteit'].astype('int32')

In [ ]:
df2['targetherstel'] = df2['targetherstel'].apply(lambda x: round(x / 10) * 10)

In [ ]:
feature = df2[['stm_oorz_code','stm_sap_melddatum','stm_geo_mld','stm_sap_meldtijd','stm_aanntpl_tijd','stm_techn_mld','stm_prioriteit']]
target = df2['targetherstel']
feats_train, feats_test, cat_train, cat_test = train_test_split(feature, target, random_state=10)

In [ ]:
target

In [ ]:
model_Tree = DecisionTreeClassifier(max_depth=50)
model_Tree.fit(feats_train, cat_train)

In [ ]:
model_Tree.score(feats_test, cat_test)

In [ ]:
X_new = df2.iloc[[-50]]
X_new =X_new[['stm_oorz_code','stm_sap_melddatum','stm_geo_mld','stm_sap_meldtijd','stm_aanntpl_tijd','stm_techn_mld','stm_prioriteit']]


In [ ]:
predictions = model_Tree.predict(X_new)
print(predictions)

In [ ]:
df2.iloc[[ -50]]